# PRÁCTICA 2: word2vec / skip-gram
### Miembros: 
- Raquel Almeida Quesada
 - Jorge Morales Llerandi

#### Cargar y tokenizar el corpus  

In [1]:
import re
from collections import Counter

with open("resources/dataset_word2vec.txt", "r", encoding="utf-8") as f:
    text = f.read().lower()
    
tokens = re.findall(r'\b[a-záéíóúüñ]+\b', text)
print("Número total de tokens:", len(tokens))

tokens


Número total de tokens: 1148


['el',
 'perro',
 'corre',
 'en',
 'el',
 'parque',
 'el',
 'perro',
 'come',
 'carne',
 'el',
 'perro',
 'bebe',
 'agua',
 'el',
 'perro',
 'duerme',
 'en',
 'la',
 'casa',
 'el',
 'perro',
 'juega',
 'con',
 'la',
 'pelota',
 'la',
 'perra',
 'corre',
 'en',
 'el',
 'parque',
 'la',
 'perra',
 'come',
 'carne',
 'la',
 'perra',
 'bebe',
 'agua',
 'la',
 'perra',
 'duerme',
 'en',
 'la',
 'casa',
 'la',
 'perra',
 'juega',
 'con',
 'la',
 'pelota',
 'el',
 'gato',
 'corre',
 'en',
 'el',
 'patio',
 'el',
 'gato',
 'come',
 'pescado',
 'el',
 'gato',
 'bebe',
 'leche',
 'el',
 'gato',
 'duerme',
 'en',
 'el',
 'sofá',
 'el',
 'gato',
 'juega',
 'con',
 'la',
 'cuerda',
 'la',
 'gata',
 'corre',
 'en',
 'el',
 'patio',
 'la',
 'gata',
 'come',
 'pescado',
 'la',
 'gata',
 'bebe',
 'leche',
 'la',
 'gata',
 'duerme',
 'en',
 'el',
 'sofá',
 'la',
 'gata',
 'juega',
 'con',
 'la',
 'cuerda',
 'el',
 'pájaro',
 'vuela',
 'sobre',
 'el',
 'árbol',
 'el',
 'pájaro',
 'canta',
 'por',
 'la',


#### Crear vocabulario y los pares (centro, contexto)

In [2]:
from itertools import chain

window_size = 2
vocab = list(set(tokens))
word_to_ix = {w: i  for i, w in enumerate(vocab)}
ix_to_word = {i: w for w, i in word_to_ix.items()}

pairs = []
for i, center in enumerate(tokens):
    for j in range(max(0, i - window_size), min(len(tokens), i + window_size + 1)):
        if i != j:
            pairs.append((center, tokens[j]))
            
print("Ejemplo de par:", pairs[:10])
            

Ejemplo de par: [('el', 'perro'), ('el', 'corre'), ('perro', 'el'), ('perro', 'corre'), ('perro', 'en'), ('corre', 'el'), ('corre', 'perro'), ('corre', 'en'), ('corre', 'el'), ('en', 'perro')]


#### Crear el dataset para Pytorch

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

class Word2VecDataset(Dataset):
    def __init__(self, pairs, word_to_ix):
        self.pairs = pairs
        self.word_to_ix = word_to_ix
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        center, context = self.pairs[idx]
        return torch.tensor(self.word_to_ix[center]), torch.tensor(self.word_to_ix[context])
    
dataset = Word2VecDataset(pairs, word_to_ix)
dataLoader = DataLoader(dataset, batch_size=64, shuffle=True)

#### Definir el modelo Skip-Gram

In [4]:
import torch.nn as nn

class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(SkipGramModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.output = nn.Linear(embedding_size, vocab_size)
        
    def forward(self, center_words):
        embeds = self.embedding(center_words)
        out = self.output(embeds)
        return out
    
embedding_dim = 50
model = SkipGramModel(len(vocab), embedding_dim)

#### Entrenamiento

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    correct = 0
    total = 0

    for center, context in dataLoader:
        optimizer.zero_grad()
        output = model(center)
        loss = criterion(output, context)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calcular accuracy (predicción correcta del contexto)
        preds = torch.argmax(output, dim=1)
        correct += (preds == context).sum().item()
        total += context.size(0)

    avg_loss = total_loss / len(dataLoader)
    accuracy = 100 * correct / total
    print(f"Época {epoch+1} | Pérdida media: {avg_loss:.4f} | Precisión: {accuracy:.2f}%")


Época 1 | Pérdida media: 5.9259 | Precisión: 0.65%
Época 2 | Pérdida media: 5.6821 | Precisión: 2.44%
Época 3 | Pérdida media: 5.4817 | Precisión: 4.67%
Época 4 | Pérdida media: 5.3011 | Precisión: 7.17%
Época 5 | Pérdida media: 5.1397 | Precisión: 8.72%
Época 6 | Pérdida media: 4.9993 | Precisión: 9.81%
Época 7 | Pérdida media: 4.8690 | Precisión: 10.66%
Época 8 | Pérdida media: 4.7569 | Precisión: 11.40%
Época 9 | Pérdida media: 4.6480 | Precisión: 12.04%
Época 10 | Pérdida media: 4.5480 | Precisión: 13.00%


#### Explorar los embeddings (vecinos más cercanos)

In [7]:
import torch.nn.functional as F

def get_embedding(word):
    idx = word_to_ix[word]
    return model.embedding.weight[idx]  # <-- aquí está bien (sin la 's')

def nearest(word, top_n=5):
    word_emb = get_embedding(word)
    sims = F.cosine_similarity(word_emb.unsqueeze(0), model.embedding.weight)  # <-- corregido
    best = torch.topk(sims, top_n+1).indices.tolist()[1:]  # quitamos la propia palabra
    return [ix_to_word[i] for i in best]

print("Vecinos de 'parís':", nearest("parís"))


Vecinos de 'parís': ['él', 'gato', 'come', 'al', 'son']


#### Pruebas de analogías

In [9]:
def analogy(w1, w2, w3, top_n=1):
    emb = get_embedding(w2) - get_embedding(w1) + get_embedding(w3)
    sims = F.cosine_similarity(emb.unsqueeze(0), model.embedding.weight)
    best = torch.topk(sims, top_n+3).indices.tolist()
    result = [ix_to_word[i] for i in best if ix_to_word[i] not in [w1, w2, w3]][:top_n]
    return result

print("parís : francia :: madrid : ?", analogy("parís", "francia", "madrid"))


parís : francia :: madrid : ? ['reina']
